In [1]:

import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import xgboost as xgb

from eegsc.preprocessing.filters import BandPassFilter
from eegsc.preprocessing.spectrum import SpectrumTransformer
from eegsc.utils.io import read_raw
from eegsc.utils.path import get_data_path
from eegsc.utils.experiments import create_spectrum_dataset
from eegsc.utils.cross_val import train_test_split, cross_val_score

In [2]:
start_time = 5
signal_duration = 10 - start_time
bandpass_filter = BandPassFilter(signal_duration=signal_duration)
spectrum_transformer = SpectrumTransformer(psd_method='periodogram',
                                           signal_duration=signal_duration,
                                           features_set='iaf_paf_psd_trp')

In [3]:
data = read_raw(os.path.join(get_data_path(), 'raw', '1st_Day.mat'),
                # data_types=['left_real', 'right_real',
                #             'left_im1', 'right_im1',
                #             'left_im2', 'right_im2',
                #             'left_quasi', 'right_quasi'])
                # data_types=['left_quasi', 'right_quasi'])
                # data_types=['left_im2', 'right_im2'])
                # data_types=['left_im1', 'right_im1'])
                data_types=['left_real', 'right_real'])

In [4]:
statistics, labels, person_idxs = create_spectrum_dataset(
    data, bandpass_filter, spectrum_transformer, start_time=start_time, is_len_equal=True)
statistics.shape, labels.shape, person_idxs.shape

before: [2.87620118e-11 2.51151235e-10 1.09939363e-09 ... 7.65832621e-07
 7.93657323e-07 8.16755430e-07]
after: [5.54390080e-07 5.50807535e-07 5.45864806e-07 ... 5.55808784e-07
 5.57002605e-07 5.56493272e-07]
before: [ 2.87620118e-11  2.51012478e-10  1.09790760e-09 ... -8.73010567e-07
 -1.03144155e-06 -1.18670462e-06]
after: [-3.97142319e-07 -5.10560035e-07 -6.19559384e-07 ... -3.54573880e-08
 -1.59058909e-07 -2.79801692e-07]
before: [1.13092992e-10 9.81835759e-10 4.26586888e-09 ... 5.55163824e-06
 5.68296660e-06 5.73819765e-06]
after: [4.03968326e-06 4.06106687e-06 4.03934306e-06 ... 3.71103197e-06
 3.86481104e-06 3.97438818e-06]
before: [ 1.45252722e-09  1.22561737e-08  5.13464314e-08 ... -1.18767549e-05
 -1.18997813e-05 -1.12980644e-05]
after: [-4.98310176e-06 -4.10736057e-06 -3.09065788e-06 ... -6.30091861e-06
 -6.11475342e-06 -5.66604454e-06]
before: [4.35559732e-11 3.63646716e-10 1.50747729e-09 ... 3.79099384e-06
 3.64546125e-06 3.48617591e-06]
after: [1.05051033e-06 9.57647448e-

((569, 320), (569,), (569,))

In [6]:
metrics = cross_val_score(data=statistics,
                          labels=labels,
                          person_idxs=person_idxs,
                          model=xgb.XGBClassifier(random_state=0),
                          metric=accuracy_score,
                          n_test_persons=3,
                          random_state=0)
metrics

 20%|██        | 1/5 [00:00<00:01,  2.46it/s]

score [ 2 11  3]: 0.7155172413793104


 40%|████      | 2/5 [00:00<00:00,  3.21it/s]

score [10  0  4]: 0.6134453781512605


 60%|██████    | 3/5 [00:00<00:00,  3.47it/s]

...

 80%|████████  | 4/5 [00:01<00:00,  3.58it/s]

score [12  6  9]: 0.4953271028037383


100%|██████████| 5/5 [00:01<00:00,  3.45it/s]

score [13  8  1]: 0.6371681415929203


,test_persons,train_metrics,test_metrics
0,"[2, 11, 3]",1.0,0.715517
1,"[10, 0, 4]",1.0,0.613445
2,"[7, 5, 14]",1.0,0.535088
3,"[12, 6, 9]",1.0,0.495327
4,"[13, 8, 1]",1.0,0.637168


In [8]:
def pca_preproc(x_train, x_test):
    pca = PCA(n_components=.9, svd_solver='full').fit(x_train)
    return pca.transform(x_train), pca.transform(x_test)

metrics = cross_val_score(data=statistics,
                          labels=labels,
                          person_idxs=person_idxs,
                          model=xgb.XGBClassifier(random_state=0),
                          metric=accuracy_score,
                          n_test_persons=3,
                          random_state=0,
                          data_preprocessor=pca_preproc)
metrics

 20%|██        | 1/5 [00:00<00:01,  3.33it/s]

score [ 2 11  3]: 0.5517241379310345
score [10  0  4]: 0.5042016806722689


 60%|██████    | 3/5 [00:00<00:00,  4.69it/s]

score [ 7  5 14]: 0.5263157894736842
score [12  6  9]: 0.45794392523364486


100%|██████████| 5/5 [00:01<00:00,  4.68it/s]

score [13  8  1]: 0.5752212389380531


,test_persons,train_metrics,test_metrics
0,"[2, 11, 3]",1.0,0.551724
1,"[10, 0, 4]",1.0,0.504202
2,"[7, 5, 14]",1.0,0.526316
3,"[12, 6, 9]",1.0,0.457944
4,"[13, 8, 1]",1.0,0.575221
